In [15]:
# Importing required libraries

import plotly.graph_objects as go
from ipywidgets import FloatSlider, Button, HBox, VBox
import pyvista as pv
import numpy as np

In [16]:
# Load the 3D dataset

dataset = pv.read('mixture.vti')


In [17]:
# Verifying the scalar field name

print("Available arrays:", dataset.point_data.keys())

Available arrays: ['ImageFile']


In [18]:
# Extracting the scalar field

scalar_field_name = 'ImageFile'
scalars = dataset.point_data[scalar_field_name]

In [19]:
# Preparing the data

dims = dataset.dimensions
data_3d = scalars.reshape(dims, order='F')
data_flattened = data_3d.flatten()
data_min, data_max = data_flattened.min(), data_flattened.max()

In [20]:
# Generating grid coordinates

x_coords = np.arange(dims[0])
y_coords = np.arange(dims[1])
z_coords = np.arange(dims[2])
X, Y, Z = np.meshgrid(x_coords, y_coords, z_coords, indexing='ij')
X, Y, Z = X.flatten(), Y.flatten(), Z.flatten()

In [21]:
# Creating initial isosurface trace

initial_isovalue = 0.0
isosurface_trace = go.Isosurface(
    x = X,
    y = Y,
    z = Z,
    value = data_flattened,
    isomin = initial_isovalue,
    isomax = initial_isovalue,
    surface = dict(count=1),
    colorscale = 'plasma',
    caps = dict(x_show=False, y_show=False, z_show=False),
    colorbar = dict(title = 'Value')
)

In [22]:
# Initialize FigureWidget for isosurface

isosurface_fig = go.FigureWidget(data=[isosurface_trace])
isosurface_fig.update_layout(
    title = 'Isosurface',
    scene = dict(aspectmode='manual', aspectratio=dict(x=1, y=1, z=1))
)

FigureWidget({
    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}, 'z': {'show': False}},
              'colorbar': {'title': {'text': 'Value'}},
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'isomax': 0.0,
              'isomin': 0.0,
              'surface': {'count': 1},
              'type': 'isosurface',
              'uid': '46bf1ad2-b42a-4f5c-b154-f981d3103fea',
              'value': {'bdata': ('2GgnvS5Dp7weGJu7Ybe3ulPtP7x05h' ... 'wNv0BBO79AfGG/y1Nxv4fFW7/Tfzq/'),
                        'dtype': 'f4'},
              'x': {'bda

In [23]:
# Initializing Histogram

histogram_trace = go.Histogram(
    x = data_flattened,
    nbinsx = 50,
    marker = dict(color = 'blue')
)
histogram_fig = go.FigureWidget(data=[histogram_trace])
histogram_fig.update_layout(
    title = 'Histogram',
    xaxis = dict(title = "Value", range = [data_min, data_max]),
    yaxis = dict(title = "Count")
)

FigureWidget({
    'data': [{'marker': {'color': 'blue'},
              'nbinsx': 50,
              'type': 'histogram',
              'uid': '88ec37ad-97b8-4f17-8b44-b9853deb6387',
              'x': {'bdata': ('2GgnvS5Dp7weGJu7Ybe3ulPtP7x05h' ... 'wNv0BBO79AfGG/y1Nxv4fFW7/Tfzq/'),
                    'dtype': 'f4'}}],
    'layout': {'template': '...',
               'title': {'text': 'Histogram'},
               'xaxis': {'range': [-0.9935540556907654, 0.43280163407325745], 'title': {'text': 'Value'}},
               'yaxis': {'title': {'text': 'Count'}}}
})

In [24]:
# Creating widgets

isovalue_slider = FloatSlider(
    value = initial_isovalue,
    min = data_min,
    max = data_max,
    step = 0.1, # adjust this according to the need of the hour.
    description = 'Isovalue:',
    continuous_update = False
)
reset_button = Button(description='Reset')

In [25]:
# Update function

def update_visualizations(change):
    x = change['new']
    # update isosurface
    isosurface_fig.data[0].isomin = x
    isosurface_fig.data[0].isomax = x

    # update histogram
    lower_bound = max(data_min, x - 0.25)
    upper_bound = min(data_max, x + 0.25)
    mask = (data_flattened >= lower_bound) & (data_flattened <= upper_bound)
    filtered_data = data_flattened[mask]
    histogram_fig.data[0].x = filtered_data
    histogram_fig.update_layout(xaxis=dict(range=[x-0.25, x+0.25]))

In [26]:
# Reset function

def reset_visualizations(button):
    isovalue_slider.value = initial_isovalue
    isosurface_fig.data[0].isomin = initial_isovalue
    isosurface_fig.data[0].isomax = initial_isovalue
    histogram_fig.update_traces(x=data_flattened)
    histogram_fig.update_layout(xaxis=dict(range=[data_min, data_max]))

In [27]:
# Connecting widgets

isovalue_slider.observe(update_visualizations, names='value')
reset_button.on_click(reset_visualizations)

In [28]:
# Displaying interface

display(VBox([
    HBox([isosurface_fig, histogram_fig]),
    HBox([isovalue_slider, reset_button])
]))

    'data': [{'caps': {'x': {'show': False}, 'y': {'show': False}…